## Constants

List of all constants to be used throughout the application.

In [2]:
PROJECT_PATH = '.'


# Dataset Labels
LABEL_NON_SUICIDAL = 0
LABEL_SUICIDAL = 1


# Root Datasets Path
DATASETS_PATH = f'{PROJECT_PATH}/datasets'


# Source Datasets

DATASETS_DOWNLOAD_PATH = f'{DATASETS_PATH}/downloads'
DATASETS_EXPORT_PATH = f'{DATASETS_PATH}/export'

TDA_DATASET_URL = "https://www.kaggle.com/datasets/thedevastator/c-ssrs-labeled-suicidality-in-500-anonymized-red"
TDA_DATASET_EXPORT_PATH = f'{DATASETS_EXPORT_PATH}/tda-dataset.csv'

AG_DATASET_URL = "https://www.kaggle.com/datasets/amangoyl/reddit-dataset-for-multi-task-nlp"
AG_DATASET_EXPORT_PATH = f'{DATASETS_EXPORT_PATH}/ag-dataset.csv'

IMS_DATASET_URL = "https://www.kaggle.com/datasets/imeshsonu/suicideal-phrases"
IMS_DATASET_EXPORT_PATH = f'{DATASETS_EXPORT_PATH}/ims-dataset.csv'

LAX_DATASET_URL = "https://raw.githubusercontent.com/laxmimerit/twitter-suicidal-intention-dataset/master/twitter-suicidal_data.csv"
LAX_DATASET_EXPORT_PATH = f'{DATASETS_EXPORT_PATH}/lax-dataset.csv'

MSH_DATASET_URL = "https://www.kaggle.com/datasets/mohanedmashaly/suicide-notes"
MSH_DATASET_EXPORT_PATH = f'{DATASETS_EXPORT_PATH}/msh-dataset.csv'


NTL_DATASET_URL = "https://www.kaggle.com/datasets/natalialech/suicidal-ideation-on-twitter"
NTL_DATASET_EXPORT_PATH = f'{DATASETS_EXPORT_PATH}/ntl-dataset.csv'

SOURCE_DATASETS = [
  TDA_DATASET_EXPORT_PATH,
  AG_DATASET_EXPORT_PATH,
  IMS_DATASET_EXPORT_PATH,
  LAX_DATASET_EXPORT_PATH,
  MSH_DATASET_EXPORT_PATH,
  NTL_DATASET_EXPORT_PATH
]

# Final Cleaned Dataset
CLEANED_DATASET_PATH = f'{DATASETS_EXPORT_PATH}/final-cleaned-dataset.csv'

## Setup Environment

In [3]:
import os
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

try:
    os.chdir(PROJECT_PATH)
    print(f"WORKING PATH: {PROJECT_PATH}")
except OSError:
    print("Error: Can't change the Current Working Directory")

WORKING PATH: .


In [3]:
!pip install opendatasets
!pip install pandas swifter
!pip install -qqq unidecode contractions pyspellchecker wordninja symspellpy
!pip install spacy matplotlib seaborn
!pip install keras tensorflow-cpu
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

!python -m spacy download en_core_web_sm

^C
Traceback (most recent call last):
  File "/home/sukanta/miniconda3/bin/pip", line 5, in <module>
    from pip._internal.cli.main import main
  File "/home/sukanta/miniconda3/lib/python3.11/site-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/home/sukanta/miniconda3/lib/python3.11/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/home/sukanta/miniconda3/lib/python3.11/site-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/home/sukanta/miniconda3/lib/python3.11/site-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/home/sukanta/miniconda3/lib/python3.11/site-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging imp

In [4]:
import opendatasets as od
import pandas as pd
import numpy as np
import ast
import spacy
import unidecode
import contractions as contract
import re
import wordninja
import collections
import pkg_resources
from spellchecker import SpellChecker
from symspellpy import SymSpell, Verbosity
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
import seaborn as sns
from transformers import pipeline
from tqdm import tqdm

tqdm.pandas()

from swifter import set_defaults
set_defaults(
    dask_threshold=1,
    scheduler="processes",
    allow_dask_on_strings=True,
    progress_bar=True,
)


2023-08-07 15:22:51.844195: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 15:22:52.733989: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-07 15:22:53.728571: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-07 15:22:53.867322: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

## Data Source: thedevastator

https://www.kaggle.com/datasets/thedevastator/c-ssrs-labeled-suicidality-in-500-anonymized-red

### Pull Data

In [5]:
od.download(TDA_DATASET_URL, DATASETS_DOWNLOAD_PATH)
tda_data = pd.read_csv(f'{DATASETS_DOWNLOAD_PATH}/c-ssrs-labeled-suicidality-in-500-anonymized-red/500_anonymized_Reddit_users_posts_labels - 500_anonymized_Reddit_users_posts_labels.csv')
tda_data

Skipping, found downloaded files in "./datasets/downloads/c-ssrs-labeled-suicidality-in-500-anonymized-red" (use force=True to force download)


,User,Post,Label
0,user-0,"['Its not a viable option, and youll be leavin...",Supportive
1,user-1,['It can be hard to appreciate the notion that...,Ideation
2,user-2,"['Hi, so last night i was sitting on the ledge...",Behavior
3,user-3,['I tried to kill my self once and failed badl...,Attempt
4,user-4,['Hi NEM3030. What sorts of things do you enjo...,Ideation
...,...,...,...
495,user-495,"['Its not the end, it just feels that way. Or ...",Supportive
496,user-496,"['It was a skype call, but she ended it and Ve...",Indicator
497,user-497,['That sounds really weird.Maybe you were Dist...,Supportive
498,user-498,['Dont know there as dumb as it sounds I feel ...,Attempt


### Analyze data based on the labels

In [3]:
tda_posts = tda_data.copy(deep=True)

def tda_fix_post(text):
  if text.endswith("']") or text.endswith('"]'): return text
  return text + "']"

tda_posts['Post'] = tda_posts['Post'].map(tda_fix_post)
tda_posts['Post'] = tda_posts['Post'].apply(ast.literal_eval)
tda_posts['Post'] = tda_posts['Post'].map(lambda post: " ".join( post ))


tda_labels = tda_posts['Label'].unique()
for tda_label in tda_labels:
  items = tda_posts[tda_posts['Label']==tda_label]
  items = items.sample(n = 10)

  print(f'LABEL: {tda_label}')
  for index in items['Post'].index:
    print(f"- {items['Post'][index]}\n")

  print()

tda_posts

NameError: name 'tda_data' is not defined

### Findings

- Based on the above samples, we may conclude that the "Attempt" label clearly gives the indication of sucidilaty.
- Also, it looks like the "Ideation" label is for the state of a person when he/she is tending towards sucidilaty. (Ideation is just before attempt)

So, for this dataset we'll use both "Attempt" & "Ideation" as suicidal. And the rest are non suicidal.

### Primary Cleanup & Save Dataset

We'll clean the dataset and finally transform it to support as a training data for **suicidal-electra** model.

In [7]:
tda_export = tda_posts[['Post', 'Label']].copy()

tda_export['Label'] = tda_export['Label'].map(lambda label: LABEL_SUICIDAL if (label == 'Ideation' or label == 'Attempt') else LABEL_NON_SUICIDAL)
tda_export = tda_export.rename(columns={"Post": "text", "Label": "class"})

tda_export.to_csv(TDA_DATASET_EXPORT_PATH, index=False)

tda_export

,text,class
0,"Its not a viable option, and youll be leaving ...",0
1,It can be hard to appreciate the notion that y...,1
2,"Hi, so last night i was sitting on the ledge o...",0
3,I tried to kill my self once and failed badly ...,1
4,Hi NEM3030. What sorts of things do you enjoy ...,1
...,...,...
495,"Its not the end, it just feels that way. Or at...",0
496,"It was a skype call, but she ended it and Vent...",0
497,That sounds really weird.Maybe you were Distra...,0
498,Dont know there as dumb as it sounds I feel Hy...,1


## Data Source: ag

https://www.kaggle.com/datasets/amangoyl/reddit-dataset-for-multi-task-nlp

### Pull Data

In [8]:
od.download(AG_DATASET_URL, DATASETS_DOWNLOAD_PATH)
ag_data = pd.read_csv(f'{DATASETS_DOWNLOAD_PATH}/reddit-dataset-for-multi-task-nlp/Dataset_Suicidal_Sentiment.csv')
ag_data

Skipping, found downloaded files in "./datasets/downloads/reddit-dataset-for-multi-task-nlp" (use force=True to force download)


,Unnamed: 0,Post,Suicidal_label,Sentiment_label
0,0,Ex Wife Threatening SuicideRecently I left my ...,0,0
1,1,Am I weird I don t get affected by compliments...,1,1
2,2,Finally is almost over So I can never hear ...,1,0
3,3,i need helpjust help me im crying so hard,0,0
4,4,I m so lostHello my name is Adam and I ve b...,0,0
...,...,...,...,...
226948,227680,I sound like a dudebro but I can t handle my f...,0,0
226949,227681,Fuck my sister She is such I fucking bitch and...,1,0
226950,227682,I ve been suicidal for years and no one knowsT...,0,1
226951,227683,My boyfriend is sick so I took some Polaroids ...,1,0


### Analyze data based on the labels

----

**Suicidal Labels**

**Suicidal** - 0

**Non-Suicidal** -1

----

**Sentiment Labels**

**Negative** - 0

**Positive** - 1

**Neutral** - 2

In [9]:
# Suicidal & Negative

print("Suicidal & Negative:\n")

ag_sample = ag_data[(ag_data['Suicidal_label']==0) & (ag_data['Sentiment_label']==0)].sample(n=10)

for index in ag_sample.index:
  print(f"- {ag_sample['Post'][index]}")



# Non-Suicidal & Negative

print("\n\nNon-Suicidal & Negative:\n")

ag_sample = ag_data[(ag_data['Suicidal_label']==1) & (ag_data['Sentiment_label']==0)].sample(n=10)

for index in ag_sample.index:
  print(f"- {ag_sample['Post'][index]}")



# Suicidal & Positive

print("\n\nSuicidal & Positive:\n")

ag_sample = ag_data[(ag_data['Suicidal_label']==0) & (ag_data['Sentiment_label']==1)].sample(n=10)

for index in ag_sample.index:
  print(f"- {ag_sample['Post'][index]}")



# Non-Suicidal & Positive

print("\n\nNon-Suicidal & Positive:\n")

ag_sample = ag_data[(ag_data['Suicidal_label']==1) & (ag_data['Sentiment_label']==1)].sample(n=10)

for index in ag_sample.index:
  print(f"- {ag_sample['Post'][index]}")



# Suicidal & Neutral

print("\n\nSuicidal & Neutral:\n")

ag_sample = ag_data[(ag_data['Suicidal_label']==0) & (ag_data['Sentiment_label']==2)].sample(n=10)

for index in ag_sample.index:
  print(f"- {ag_sample['Post'][index]}")



# Non-Suicidal & Neutral

print("\n\nNon-Suicidal & Neutral:\n")

ag_sample = ag_data[(ag_data['Suicidal_label']==1) & (ag_data['Sentiment_label']==2)].sample(n=10)

for index in ag_sample.index:
  print(f"- {ag_sample['Post'][index]}")

Suicidal & Negative:

- Would you want your son to leave a suicide note I think my parents know my suicide is coming  They ve stopped asking me about my future plans  how my job hunting is going  etc  We don t speak much outside of logistics  things like  What do we need from the grocery store   Should I leave a suicide note when I punch out next month  Would that make it worse
- A cry for helpWill this ever end  I can t get all my memories out of my head  My thoughts come and take over my body  I feel frozen and my brain convinces me that following an urge will bring me closer to my  goal  even though logically I know it s not in that moment the urge tricks my logic and that becomes the new truth  I did my urge   days in a row   years almost later I m still so hurt over an ex I hate myself I hate myself  I wanna just jump off a building and finish everything  I m in pain because I m a broken person  I wanna be numb  I m so alone right now I hate myself I hate seeing myself I hate ever

### Findings

- Based on the above samples, we may conclude that the **Suicidal & Negative** is an absolute sucidal.

- Some of the other **Suicidal & Positive** & **Suicidal & Neutral** is not a complete suicidal. But, we may ignore.

So, for this dataset we'll use all items with `"Suicidal_label" = 0`. It also seems like the "sentiments" has very less corelation with suicidality. We can surely ignore it.

### Primary Cleanup & Save Dataset

We'll clean the dataset and finally transform it to support as a training data for **suicidal-electra** model.

In [10]:
ag_export = ag_data[['Post', 'Suicidal_label']].copy()
ag_export = ag_export.rename(columns={"Post": "text", "Suicidal_label": "class"})

ag_export['class'] = ag_export['class'].map(lambda label: LABEL_SUICIDAL if (label == 0) else LABEL_NON_SUICIDAL)

ag_export.to_csv(AG_DATASET_EXPORT_PATH, index=False)

ag_export

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,1
1,Am I weird I don t get affected by compliments...,0
2,Finally is almost over So I can never hear ...,0
3,i need helpjust help me im crying so hard,1
4,I m so lostHello my name is Adam and I ve b...,1
...,...,...
226948,I sound like a dudebro but I can t handle my f...,1
226949,Fuck my sister She is such I fucking bitch and...,0
226950,I ve been suicidal for years and no one knowsT...,1
226951,My boyfriend is sick so I took some Polaroids ...,0


## Data Source: imeshsonu

https://www.kaggle.com/datasets/imeshsonu/suicideal-phrases

### Pull Data

In [11]:
od.download(IMS_DATASET_URL, DATASETS_DOWNLOAD_PATH)
ims_data = pd.read_csv(f'{DATASETS_DOWNLOAD_PATH}/suicideal-phrases/Train_suicide1.csv')
ims_data

Skipping, found downloaded files in "./datasets/downloads/suicideal-phrases" (use force=True to force download)


,Tweet,Suicide
0,i hate myself so much i want to KILL myself ho...,Potential Suicide post
1,RT @DrugForumsBest: I woke up with a bag over ...,Potential Suicide post
2,i wanna fucking kill myself,Potential Suicide post
3,why do i destroy all my opportunities? am i go...,Potential Suicide post
4,u - understand what the fuck im doing wrong wi...,Potential Suicide post
...,...,...
1594,hi! just joined and i think it's brill! haha s...,Not Suicide post
1595,@WilliamSledd Love the videos William New to t...,Not Suicide post
1596,@itsanimesh I got an Openmoko FreeRunner and n...,Not Suicide post
1597,@PeoplemapsJulie I am looking for the berocca ...,Not Suicide post


### Analyze data based on the labels

In [12]:
ims_posts = ims_data.copy(deep=True)

ims_labels = ims_posts['Suicide'].unique()

for label in ims_labels:
  items = ims_posts[ims_posts['Suicide']==label]
  
  items = items.sample(n = 10)
  items = items.dropna()

  print(f'LABEL: {label}\n--------------------------------')
  for index in items.index:
    print(f"- {items['Tweet'][index]}")
    
  print()

ims_posts

LABEL: Potential Suicide post 
--------------------------------
- @Oomongoomong,,This is for u, the person who feels like they have no one, the person who doesnt see why they should be here anymore
- Sex dreams are my least favorite dreams. Like they're always weird and appear out of nowhere and then you think its… https://t.co/1fdKAwFjwE
- @_creamqueen Lady, I’m on the hill ready to fucking just die already.
- @AnitaHarrell19 @PeterParros I agree with everything except Benny. I'm tired of him. His character is boring and needs to die. IJS 😊
- Please retweet my Weed G. Board!!! The coolest adult cartoon.  I'm tired of living the worst times of my life, plea… https://t.co/jxWJLxgT9x
- I hope God takes me soon. I legit am tired of living and really just wanna die. I feel like crying for everything l… https://t.co/hmnfAKgolY
- Don’t break other people down just because they may seem “weaker” than you. If someone’s eyes are 20/20 that doesn’… https://t.co/DHtb9JZKxE
- i wanna fucking kill 

,Tweet,Suicide
0,i hate myself so much i want to KILL myself ho...,Potential Suicide post
1,RT @DrugForumsBest: I woke up with a bag over ...,Potential Suicide post
2,i wanna fucking kill myself,Potential Suicide post
3,why do i destroy all my opportunities? am i go...,Potential Suicide post
4,u - understand what the fuck im doing wrong wi...,Potential Suicide post
...,...,...
1594,hi! just joined and i think it's brill! haha s...,Not Suicide post
1595,@WilliamSledd Love the videos William New to t...,Not Suicide post
1596,@itsanimesh I got an Openmoko FreeRunner and n...,Not Suicide post
1597,@PeoplemapsJulie I am looking for the berocca ...,Not Suicide post


### Findings

- Based on the above samples, we may conclude that the "Potential Suicide post" label clearly gives an indication of suicidality.
- Also, it looks like the "Not Suicide post" label is not suicidal.

So, for this dataset, we'll use both "Attempt" & "Ideation" as suicidal. And the rest are nonsuicidal.

### Primary Cleanup & Save Dataset

We'll clean the dataset and finally transform it to support as a training data for **suicidal-electra** model.

In [13]:
ims_export = ims_posts[['Tweet', 'Suicide']].copy()

ims_export['Suicide'] = ims_export['Suicide'].map(lambda label: LABEL_SUICIDAL if (label == 'Potential Suicide post ') else LABEL_NON_SUICIDAL)

ims_export = ims_export.rename(columns={"Tweet": "text", "Suicide": "class"})

ims_export.to_csv(IMS_DATASET_EXPORT_PATH, index=False)
ims_export

,text,class
0,i hate myself so much i want to KILL myself ho...,1
1,RT @DrugForumsBest: I woke up with a bag over ...,1
2,i wanna fucking kill myself,1
3,why do i destroy all my opportunities? am i go...,1
4,u - understand what the fuck im doing wrong wi...,1
...,...,...
1594,hi! just joined and i think it's brill! haha s...,0
1595,@WilliamSledd Love the videos William New to t...,0
1596,@itsanimesh I got an Openmoko FreeRunner and n...,0
1597,@PeoplemapsJulie I am looking for the berocca ...,0


## Data Source: laxmimerit

https://raw.githubusercontent.com/laxmimerit/twitter-suicidal-intention-dataset/master/twitter-suicidal_data.csv

### Pull Data

In [14]:
od.download(LAX_DATASET_URL, DATASETS_DOWNLOAD_PATH)
lax_data = pd.read_csv(f'{DATASETS_DOWNLOAD_PATH}/twitter-suicidal_data.csv')
lax_data

Using downloaded and verified file: ./datasets/downloads/twitter-suicidal_data.csv


,tweet,intention,len
0,my diary log as of so far okay my first time w...,1,24038
1,i hate my life and wish i wasnt me very long p...,1,15726
2,my life so far i cant reach out to anyone soi ...,1,14960
3,tears in the shadow pt1 i figured out that i n...,1,12581
4,why i have depression and what it makes me fee...,1,9396
...,...,...,...
9114,,0,1
9115,l,0,1
9116,,0,1
9117,,0,1


### Analyze data based on the labels

In [15]:
lax_posts = lax_data.copy(deep=True)

lax_labels = lax_posts['intention'].unique()

for label in lax_labels:
  items = lax_posts[lax_posts['intention']==label]
  
  items = items.sample(n=10)
  items = items.dropna()

  print(f'LABEL: {label}\n--------------------------------')
  for index in items.index:
    print(f"- {items['tweet'][index]}")
    
  print()

lax_posts

LABEL: 1
--------------------------------
- i turned 18 this week i dont have any dreams or goals and waking up is just painfuli am not sayingi ama special snowflake because i know most of you on this sub go through worse stuff than me i know nobody asked for this so i dont really want to be a whiny brat but ive honestly had enoughi ama shitty person i dont reply to messages from people who are still actually trying to talk to me because it all seems pointless to me i leave my bed only if its absolutely necessary and i hate spending time with others no matter how important those people are to me i used to love art but i cant bring myself to create things anymore at the beginning of summer break i was likei am going to draw every single day for the following two months so i can form a habit id like to maintain and work hard on becoming an artist well guess what summer breaks ending today and i only sketched once and it ended up being a piece of garbage i realized if i actually wanted to

,tweet,intention,len
0,my diary log as of so far okay my first time w...,1,24038
1,i hate my life and wish i wasnt me very long p...,1,15726
2,my life so far i cant reach out to anyone soi ...,1,14960
3,tears in the shadow pt1 i figured out that i n...,1,12581
4,why i have depression and what it makes me fee...,1,9396
...,...,...,...
9114,,0,1
9115,l,0,1
9116,,0,1
9117,,0,1


### Findings

- Based on the above samples, we may conclude that the "1" label clearly gives an indication of suicidality.
- Also, it looks like the "0" label is not suicidal.

### Primary Cleanup & Save Dataset

We'll clean the dataset and finally transform it to support as a training data for **suicidal-electra** model.

In [16]:
lax_export = lax_posts[['tweet', 'intention']].copy()

lax_export['intention'] = lax_export['intention'].map(lambda label: LABEL_SUICIDAL if (label == 1) else LABEL_NON_SUICIDAL)

lax_export = lax_export.rename(columns={"tweet": "text", "intention": "class"})

lax_export.to_csv(LAX_DATASET_EXPORT_PATH, index=False)
lax_export

,text,class
0,my diary log as of so far okay my first time w...,1
1,i hate my life and wish i wasnt me very long p...,1
2,my life so far i cant reach out to anyone soi ...,1
3,tears in the shadow pt1 i figured out that i n...,1
4,why i have depression and what it makes me fee...,1
...,...,...
9114,,0
9115,l,0
9116,,0
9117,,0


## Data Source: mohanedmashaly

https://www.kaggle.com/datasets/mohanedmashaly/suicide-notes

### Pull Data

In [17]:
od.download(MSH_DATASET_URL, DATASETS_DOWNLOAD_PATH)
msh_data = pd.read_csv(f'{DATASETS_DOWNLOAD_PATH}/suicide-notes/test.csv')
msh_data

Skipping, found downloaded files in "./datasets/downloads/suicide-notes" (use force=True to force download)


,id,text
0,1,I have to put this in writing somehow so I sti...
1,2,I've made my peace with the fact that I'm goin...
2,3,I really want help now i just cant so this any...
3,4,I can't think of any reason to stay. I have no...
4,5,NaN
...,...,...
493,494,Since I was 9 I felt depressed and almost took...
494,495,I don't know why I'm reaching out now of all t...
495,496,"I dont believe in an afterlife, but i stay up ..."
496,497,"I don't think I actually will kill myself, but..."


### Findings

- The above are all suicide notes, which can be direcyly assigned as suicidal.

### Primary Cleanup & Save Dataset

We'll clean the dataset and finally transform it to support as a training data for **suicidal-electra** model.

In [4]:
msh_export = msh_data[['text']].copy()

msh_export['class'] = LABEL_SUICIDAL

msh_export.to_csv(MSH_DATASET_EXPORT_PATH, index=False)
msh_export

NameError: name 'msh_data' is not defined

## Data Source: natalialech (ntl)

https://www.kaggle.com/datasets/natalialech/suicidal-ideation-on-twitter

### Pull Data

In [5]:
od.download(NTL_DATASET_URL, DATASETS_DOWNLOAD_PATH)
ntl_data = pd.read_csv(f'{DATASETS_DOWNLOAD_PATH}/suicidal-ideation-on-twitter/suicidal ideation dataset.csv')
ntl_data

Skipping, found downloaded files in "./datasets/downloads/suicidal-ideation-on-twitter" (use force=True to force download)


,id,label,dataset
0,1608974517421985793,4,training
1,1519909372314341376,2,training
2,1608975908509192193,4,training
3,1608975532112384000,4,training
4,1608975358275223553,4,training
...,...,...,...
81514,1608976180627079168,4,test
81515,1608974539790352384,4,test
81516,1608974442717405191,4,test
81517,1608975883192397829,4,training


### Populate Tweet using ID

In [8]:
!pip install requests

In [42]:
import requests

def get_tweet_by_id(id):
  url = f'http://localhost:7373/tweet-by-id/?id={id}'
  
  try:
    response = requests.get(url)
    tweet = response.text
    return '' if tweet == None else str(tweet)

  except:
    return ''

ntl_export = ntl_data.copy().sample(10)

ntl_export['tweet'] = ntl_export.swifter.apply(lambda id: get_tweet_by_id(id))


ntl_export

Pandas Apply: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.18it/s]


,id,label,dataset,tweet
5424,1400051688648720384,1,training,NaN
49552,1608974877226336257,4,training,NaN
78460,1608974547629531138,4,training,NaN
59114,1608975026124115969,4,training,NaN
49626,1608975699947458560,4,training,NaN
12889,1608974818656780290,4,training,NaN
55839,1479017082293563392,0,test,NaN
54100,1608975003718156289,4,training,NaN
66967,1608975447106416642,4,training,NaN
172,1593025462451965954,3,test,NaN


In [21]:
ntl_export['tweet'].apply(lambda tweet: print(f'- {tweet}'))

- https://strawberry⠀.⠀

genuinely think my account is shadow-banned bc nobody gets a notification when i follow them && my pin doesn't appear on any of the hash tags when you search them up .

BUT I DON'T KNOW HOW BC I JUST MADE IT ?????
- As much progress as I have made with my depression, there are still days I wake up and I just wish I didn’t exist. I don’t think that’ll ever go away.
- http://seibertron.comさんの新しい記事が到着したよ。【自動投稿】 #トランスフォーマー #Transformers 10 of 2022 - Share with us your Top 10 Transformers Purchases of 2022
- Dunbrooke Women's Gray San Francisco 49ers Full-Zip Softshell Fleece Jacket CRIMGQV

https://amazon.com/dp/B06X9PKLPY?tag=anniecstiedemann-20…
- Tw:// mental health 

Sometimes I wish I was never born. Why did my mum choose to have another baby ? Why why
- I feel like doing you a $SOLID 

Join my friends at 
@SolidSignalAi
 and get this party going 

Don't miss it 

#NFT #RealBizz #ETH #airdrop #giveway
- hamstrings ready to end my life whike walking down the st

47835    None
618      None
47682    None
16694    None
24381    None
16524    None
19027    None
51116    None
470      None
4238     None
Name: tweet, dtype: object

### Findings - Not Feasible

After running several tests, it looks like, fetching the tweets without using API, limits its usage and making it completely unusable.
For now, we are dropping this.

In [81]:
ntl_export = ntl_data[['tweet', 'label']].copy()

ntl_export

# msh_export['class'] = LABEL_SUICIDAL

# msh_export.to_csv(MSH_DATASET_EXPORT_PATH, index=False)
# msh_export

,tweet,label
0,None,4
1,None,2
2,None,4
3,None,4
4,None,4
...,...,...
81514,None,4
81515,None,4
81516,None,4
81517,None,4


### Define Pre-Processors

In [ ]:
# Summarizer
summarize = pipeline("summarization", model="philschmid/bart-large-cnn-samsum", framework='pt')

# Create chunks

def chunks(l, n):
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]


# generate summary recursively

def recursive_summary(text, chunk_size=250, min_length=40, max_length=50):

    if len( text ) <= max_length:
      return text
  
    lines = list(map(lambda words: ' '.join(words), list(chunks(text.split(), chunk_size))))
    summaries = []
    for line in lines:

        if len(line) < 30:
            continue

        s = summarize(line, min_length=min_length, max_length=max_length)
        s = s[0]["summary_text"]
        summaries.append(s)

    text = ' '.join(summaries)

    if len(lines) <= 1:
        return text

    return recursive_summary(text, chunk_size=chunk_size, min_length=min_length, max_length=max_length)

In [ ]:
# Defining Methods

nlp = spacy.load("en_core_web_sm")

vocab = collections.Counter()

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
bigram_path = pkg_resources.resource_filename("symspellpy", "frequency_bigramdictionary_en_243_342.txt")

sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)


# Spell Check using Symspell
def fix_spelling(text):
    suggestions = sym_spell.lookup_compound(text, max_edit_distance=2)
    correctedtext = suggestions[0].term # get the first suggestion, otherwise returns original text if nothing is corrected
    return correctedtext

# Remove some important words from stopwords list
deselect_stop_words = ['no', 'not']

for w in deselect_stop_words:
    nlp.vocab[w].is_stop = False

# Remove extra whitespaces from text
def remove_whitespace(text):
    text = text.strip()
    return " ".join(text.split())

# Remove accented characters from text, e.g. café
def remove_accented_chars(text):
    text = unidecode.unidecode(text)
    return text

# Remove URL
def remove_url(text):
    return re.sub(r'http\S+', '', text)

# Removing symbols and digits
def remove_symbols_digits(text):
    return re.sub('[^a-zA-Z\s]', ' ', text)

# Removing special characters
def remove_special(text):
    return text.replace("\r", " ").replace("\n", " ").replace("    ", " ").replace('"', '')

# Fix word lengthening (characters are wrongly repeated)
def fix_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

def text_preprocessing(text, accented_chars=True, contractions=True, convert_num=True,
                       extra_whitespace=True, lemmatization=True, lowercase=True,
                       url=True, symbols_digits=True, special_chars=True,
                       stop_words=True, lengthening=True, spelling=True):
    """preprocess text with default option set to true for all steps"""
    if accented_chars == True: # remove accented characters
        text = remove_accented_chars(text)
    if contractions == True: # expand contractions
        text = contract.fix(text)
    if lowercase == True: # convert all characters to lowercase
        text = text.lower()
    if url == True: # remove URLs before removing symbols
        text = remove_url(text)
    if symbols_digits == True: # remove symbols and digits
        text = remove_symbols_digits(text)
    if special_chars == True: # remove special characters
        text = remove_special(text)
    if extra_whitespace == True: # remove extra whitespaces
        text = remove_whitespace(text)
    if lengthening == True: # fix word lengthening
        text = fix_lengthening(text)
    if spelling == True: # fix spelling
        text = fix_spelling(text)

    doc = nlp(text) # tokenise text

    clean_text = []

    # return text

    for token in doc:
        flag = True
        edit = token.text
        # remove stop words
        if stop_words == True and token.is_stop and token.pos_ != 'NUM':
            flag = False
        # exclude number words
        if convert_num == True and token.pos_ == 'NUM' and flag == True:
            flag = False
        # convert tokens to base form
        elif lemmatization == True and token.lemma_ != "-PRON-" and flag == True:
            edit = token.lemma_
        # append tokens edited and not removed to list
        if edit != "" and flag == True:
            clean_text.append(edit)
    return " ".join(clean_text)

### Load & merge all datasets

In [ ]:
ds = pd.DataFrame()

for DATASET in SOURCE_DATASETS:
  try:
    source_ds = pd.read_csv(DATASET).reset_index(drop=True)
    ds = pd.concat([ds, source_ds], axis=0)
  except:
    print(f"WARNING: Failed loading: {DATASET}")

    
print("UNCLEARED: ", ds.shape)

ds = ds.dropna(axis=0)
ds = ds.reset_index(drop=True)

print("CLEARED: ", ds.shape)

ds

### Text Propressing

We'll run the text processing pipeline for the dataset, to make sure all the textual contents are corrected and normalized, which can later be used to train suicidal-electra model.

In [ ]:
# Get word count of posts
text_len = [len(x.split()) for x in ds['text']]
pd.Series(text_len).hist(bins=80)
plt.show()
print(pd.Series(text_len).describe(percentiles=[.1,.2,.3,.4,.5,.6,.7,.8,.9]))

#### Sample Preprocessing

Initially we'll run a test on a small sample to make sure everything is working as expected.

In [ ]:
summary_threshold = 200

sample = ds.sample(n=50)

# apply summarizer to large text
print("SUMMARIZE")
sample['summary'] = sample['text'].progress_apply(lambda text : text if len(text) <= summary_threshold else recursive_summary(text, chunk_size=300, min_length=50, max_length=100) )

# apply text pre-processing
print("PRE-PROCESS")
sample['cleaned'] = sample['summary'].sample(n=30).swifter.apply(lambda row: text_preprocessing(row))

sample


In [ ]:
for index in sample.index:
  print(f"RAW: {sample['text'][index]}\n")
  print(f"SMR: {sample['summary'][index]}\n")
  print(f"CLN: {sample['cleaned'][index]}\n")
  print("-------\n")

#### Full Dataset Pre-Processing

Preprocess the complete dataset.

In [ ]:
export_ds = ds.copy()

# WHEN DEVELOPING, USE SMALLER SAMPLE.
export_ds = export_ds.sample(5000)

ds['cleaned'] = export_ds['text'].swifter.apply(lambda row: text_preprocessing(row))

export_ds

### **Remove Irrelevant Words**

In [ ]:
# Cleanup of irrelevant words.

tokenizer = Tokenizer()
tokenizer.fit_on_texts(export_ds['text'])
word_freq = pd.DataFrame(tokenizer.word_counts.items(), columns=['word','count']).sort_values(by='count', ascending=False)

In [ ]:
# Plot bar graph for word frequency
plt.figure(figsize=(16, 8))
sns.barplot(x='count',y='word',data=word_freq.iloc[:30])
plt.title('Most Frequent Words')
plt.xlabel("Frequency")
plt.ylabel("Word")
plt.show()

### **Remove Anomalous Words**
Remove anomalous words that may adversely affect the model by unintended bias.

In [ ]:
# Removed anomalous words
export_ds['text'] = export_ds['text'].replace('filler', '')

export_ds

### Remove 0 Word Text

In [ ]:
# Remove rows with text length 0
export_ds = export_ds[export_ds['text'].apply(lambda x: len(x.split())!=0)]
export_ds.reset_index(drop=True, inplace=True)

export_ds.shape

### **Remove outliers**
Text with large textual content can be ignored.

In [ ]:
# Get word count of posts
text_len = [len(x.split()) for x in export_ds['text']]
pd.Series(text_len).hist(bins=60)
plt.show()
print(pd.Series(text_len).describe())

In [ ]:
# Subset dataset to obtain rows with less than or equal to 62 words

MAX_WORD_COUNT = 59

export_ds = export_ds[export_ds['text'].apply(lambda x: len(x.split()) <= MAX_WORD_COUNT)]
export_ds.reset_index(drop=True, inplace=True)

export_ds

### **Save Cleaned Dataset**

Save the final cleaned dataset to a csv file.

In [ ]:
export_ds.to_csv(CLEANED_DATASET_PATH, index=False)